In [1]:
import cv2
import mediapipe as mp
import numpy as np
import yaml

# Initialize mediapipe solutions
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

# Define indices for specific facial features
FACIAL_KEYPOINTS_INDICES = {
    "eyes": list(range(33, 133)),  # Indices for eyes (approx)
    "eyebrows": list(range(70, 80)) + list(range(100, 110)),  # Indices for eyebrows (approx)
    "lips": list(range(287, 317)),  # Indices for lips (approx)
    "nose": list(range(195, 230)),  # Indices for nose (approx)
}

# Collect all indices and ensure unique indices
FACIAL_KEYPOINTS_INDICES = list(set(sum(FACIAL_KEYPOINTS_INDICES.values(), [])))

def get_keypoints(image_path):
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=2, min_detection_confidence=0.5)
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)
    pose = mp_pose.Pose(static_image_mode=True, model_complexity=2, min_detection_confidence=0.5)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Left and Right Hand keypoints
    hand_results = hands.process(image_rgb)
    left_hand_kp, right_hand_kp = np.zeros((21, 3)), np.zeros((21, 3))
    if hand_results.multi_hand_landmarks:
        for hand_landmarks in hand_results.multi_hand_landmarks:
            if hand_landmarks:
                hand_keypoints = np.array(
                    [[lm.x * width, lm.y * height, lm.z] for lm in hand_landmarks.landmark])
                if len(hand_keypoints) == 21:
                    if hand_results.multi_handedness[0].classification[0].label == 'Left':
                        left_hand_kp = hand_keypoints
                    else:
                        right_hand_kp = hand_keypoints
    
    # Face keypoints
    face_results = face_mesh.process(image_rgb)
    face_kp = np.zeros((70, 3))
    if face_results.multi_face_landmarks:
        for face_landmarks in face_results.multi_face_landmarks:
            face_keypoints = np.array(
                [[lm.x * width, lm.y * height, lm.z] for lm in face_landmarks.landmark])
            selected_keypoints = face_keypoints[FACIAL_KEYPOINTS_INDICES]
            if len(selected_keypoints) >= 70:  # Ensure at least 70 keypoints are available
                face_kp[:70] = selected_keypoints[:70]  # Taking the first 70 keypoints from the selected ones
    
    # Pose keypoints
    pose_results = pose.process(image_rgb)
    pose_kp = np.zeros((23, 3))
    if pose_results.pose_landmarks:
        pose_keypoints = np.array(
            [[lm.x * width, lm.y * height, lm.z] for lm in pose_results.pose_landmarks.landmark])
        if len(pose_keypoints) == 33:  # Mediapipe pose has 33 points
            pose_kp[:23] = pose_keypoints[:23]  # Taking only the first 23 keypoints

    return left_hand_kp, right_hand_kp, face_kp, pose_kp



In [17]:
image_path = r'F:\SIGN_LANG\edn\EverybodyDanceNow\towhid\data\train\original_frames'
image_name = 'frame020001'

# Example function call, replace with actual image path
left_hand_kp, right_hand_kp, face_kp, pose_kp = get_keypoints(image_path + '/' + image_name + '.jpg')

c:\Users\mdtow\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [18]:
# # Prepare data for YAML format
# data_lh = {
#     'hand_left_0': {
#         "!!opencv-nd-matrix": {
#             'sizes': [1, 21, 3],
#             'dt': 'f',
#             'data': left_hand_kp.flatten().tolist()
#         }
#     }
# }

# data_rh = {
#     'hand_right_0': {
#         'sizes': [1, 21, 3],
#         'dt': 'f',
#         'data': right_hand_kp.flatten().tolist()
#     }
# }

# data_face = {
#     'face_0': {
#         'sizes': [1, 70, 3],
#         'dt': 'f',
#         'data': face_kp.flatten().tolist()
#     }
# }

# data_pose = {
#     "pose_0": {
#         "sizes": [1, 23, 3],
#         "dt": "f",
#         "data": pose_kp.flatten().tolist()
#     }
# }

# # Add the YAML version header
# yaml_str = "%YAML:1.0\n"

# yaml_data_pose = yaml.dump(data_pose, default_flow_style=None)
# yaml_str_pose = yaml_str + yaml_data_pose

# yaml_data_face = yaml.dump(data_face, default_flow_style=None)
# yaml_str_face = yaml_str + yaml_data_face

# yaml_data_lh = yaml.dump(data_lh, default_flow_style=None)
# yaml_str_lh = yaml_str + yaml_data_lh

# yaml_data_rh = yaml.dump(data_rh, default_flow_style=None)
# yaml_str_rh = yaml_str + yaml_data_rh

yaml_str_lh = f"""%YAML:1.0
face_0: !!opencv-nd-matrix
  sizes: {[1, 21, 3]}
  dt: f
  data: {left_hand_kp.flatten().tolist()}
"""

yaml_str_rh = f"""%YAML:1.0
face_0: !!opencv-nd-matrix
  sizes: {[1, 21, 3]}
  dt: f
  data: {right_hand_kp.flatten().tolist()}
"""

yaml_str_face = f"""%YAML:1.0
face_0: !!opencv-nd-matrix
  sizes: {[1, 70, 3]}
  dt: f
  data: {face_kp.flatten().tolist()}
"""

yaml_str_pose = f"""%YAML:1.0
face_0: !!opencv-nd-matrix
  sizes: {[1, 23, 3]}
  dt: f
  data: {pose_kp.flatten().tolist()}
"""

keypoints_path = r'F:\SIGN_LANG\edn\EverybodyDanceNow\towhid\data\train\keypoints'

# Write data to YAML file
with open(keypoints_path + '\\' + image_name + '_hand_left.yml', 'w') as file:
    file.write(yaml_str_lh)

with open(keypoints_path + '\\' + image_name + '_hand_right.yml', 'w') as file:
    file.write(yaml_str_rh)

with open(keypoints_path + '\\' + image_name + '_face.yml', 'w') as file:
    file.write(yaml_str_face)

with open(keypoints_path + '\\' + image_name + '_pose.yml', 'w') as file:
    file.write(yaml_str_pose)

print('Data written to YAML files successfully!')

Data written to YAML files successfully!


In [12]:
data_list = [1.02090930e+03, 1.51816254e+02, 6.56632066e-01]
size_list = [1, 70, 3]

# Construct the YAML content dynamically
yaml_content = f"""%YAML:1.0
face_0: !!opencv-nd-matrix
  sizes: {size_list}
  dt: f
  data: {data_list}
"""

# Save to a .yml file
with open('output.yml', 'w') as file:
    file.write(yaml_content)

print("YAML file created successfully.")


YAML file created successfully.
